# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt

PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)

/home/eddyod/programming/pipeline/src


In [2]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import affine_transform_point, get_affine_transformation, \
    fetch_coms, list_coms, compute_affine_transformation, affine_transform_volume
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
    read_image, write_image


In [3]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def get_coms(animal, scaling_factor=1):
    """
    Fetches the COMs from disk. The data is stored in micrometers.
    """
    
    coms = {}
    dirpath = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/{animal}/com'
    if not os.path.exists(dirpath):
        return coms
    files = sorted(os.listdir(dirpath))
    for file in files:
        structure = Path(file).stem
        filepath = os.path.join(dirpath, file)
        com = np.loadtxt(filepath)
        com /= scaling_factor 
        coms[structure] = com
    return coms


In [4]:
moving_name = 'AtlasV8'
fixed_name = 'Allen'
moving_all = get_coms(moving_name, scaling_factor=1)
fixed_all = get_coms(fixed_name, scaling_factor=1)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('10N_L','10N_R')
good_keys = ['SC', 'IC']
good_keys.extend(['AP'])
good_keys.extend(['3N_L','3N_R'])
good_keys.extend(['4N_L','4N_R'])
good_keys.extend(['5N_L','5N_R'])
good_keys.extend(['6N_L','6N_R'])
good_keys.extend(['7N_L','7N_R'])
#good_keys.extend(['7n_L','7n_R']) # upped the rms a bit
good_keys.extend(['Amb_L', 'Amb_R'])
good_keys.extend(['DC_L', 'DC_R'])
#good_keys.extend(['LC_L', 'LC_R'])
good_keys.extend(['LRt_L', 'LRt_R'])
#good_keys.extend(['PBG_L','PBG_R'])
good_keys.extend(['SNC_L','SNC_R'])
#good_keys.extend(['SNR_L','SNR_R']) # upped a bit
good_keys.extend(['Sp5C_L','Sp5C_R']) # improved 5
good_keys.extend(['Sp5I_L','Sp5I_R']) # improved 4
good_keys.extend(['Sp5O_L','Sp5O_R']) # improved 1
good_keys.extend(['VLL_L','VLL_R']) # improved 2
common_keys = set(good_keys) - set(bad_keys)
#good_keys.extend(['',''])

#good_keys = common_keys
moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(good_keys))
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(np.round(moving_src[1]))
print(np.round(fixed_src[1]))

df_list = []
error = []
transformed_dict = {}
for structure in common_keys:    
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = affine_transform_point(moving0, transformation_matrix)
    difference = [a - b for a, b in zip(transformed, fixed0)]
    diff_moving_fixed = [a - b for a, b in zip(moving0, fixed0)]
    
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), np.round(diff_moving_fixed), ss]
    #print(row)
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
rms = sum(error)/len(df_list)
print(f'RMS: {rms} observations: {len(df_list)}')

29
[12196.  3948.  4373.]
[10378.  2135.  5695.]
RMS: 203.74276574783948 observations: 29


In [ ]:
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 DB to Allen RMS: 237.06805950085737 observations: 37
# MD585 to MD594 152.06606097021333 observations: 51
# MD585 to Allen 263.31435 observations: 37
# MD589 to Allen 260.02 observations: 37
# MD594 to Allen 250.79 observations: 37

In [5]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#comtfm = np.array([824.6051918494063, 80.83004570523167, 363.4390121956811])
transformed_structure = affine_transform_point(com, transformation_matrix)

print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
print(f'{moving_name} {structure} apply trans {np.round(transformed_structure/1)}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
diff = transformed_structure - fixed_all[structure]
#comdiff = comtfm - fixed_all[structure]

print(f'{moving_name}->{fixed_name} error {structure} {diff}')
#print(f'{moving_name}->{fixed_name} tfm error {structure} {comdiff}')

AtlasV8 SC non trans [10893.  3975.  4492.]
AtlasV8 SC apply trans [9136. 2077. 5765.]
Allen SC [9162. 2120. 5695.]
AtlasV8->Allen error SC [-25.40064053 -43.38456785  70.05726369]


In [6]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'diff_M_F', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)
#20	3N_R	[1079.0, 531.0, 485.0]	[910.0, 380.0, 17.0]	[873.0, 358.0, 298.0]	[-37.0, -21.0, 281.0]	284.3851
#7	4N_L	[1135.0, 529.0, 423.0]	[959.0, 378.0, 544.0]	[923.0, 362.0, 356.0]	[-36.0, -16.0, -188.0]	192.0327

,structure,AtlasV8,Allen,transformed,difference,diff_M_F,sumsquares
Index,,,,,,,
20,IC,"[12196.0, 3948.0, 4373.0]","[10378.0, 2135.0, 5695.0]","[10379.0, 2142.0, 5638.0]","[2.0, 7.0, -57.0]","[1818.0, 1813.0, -1322.0]",57.5855
14,5N_R,"[12057.0, 6242.0, 5805.0]","[10196.0, 5288.0, 7295.0]","[10166.0, 5338.0, 7253.0]","[-30.0, 50.0, -42.0]","[1861.0, 953.0, -1490.0]",72.2465
0,4N_L,"[11380.0, 5280.0, 4211.0]","[9588.0, 3779.0, 5436.0]","[9546.0, 3847.0, 5415.0]","[-42.0, 68.0, -20.0]","[1792.0, 1501.0, -1225.0]",82.1181
1,SNC_R,"[10161.0, 6162.0, 5641.0]","[8357.0, 5126.0, 7095.0]","[8359.0, 5054.0, 7049.0]","[2.0, -72.0, -46.0]","[1804.0, 1036.0, -1454.0]",85.4897
13,SC,"[10893.0, 3975.0, 4492.0]","[9162.0, 2120.0, 5695.0]","[9136.0, 2077.0, 5765.0]","[-25.0, -43.0, 70.0]","[1731.0, 1855.0, -1203.0]",86.2290
4,LRt_L,"[14496.0, 7426.0, 3359.0]","[12347.0, 6993.0, 4395.0]","[12425.0, 6922.0, 4407.0]","[78.0, -71.0, 11.0]","[2149.0, 433.0, -1036.0]",106.2206
27,6N_L,"[12614.0, 6262.0, 4081.0]","[10774.0, 5217.0, 5293.0]","[10683.0, 5260.0, 5254.0]","[-91.0, 44.0, -39.0]","[1840.0, 1045.0, -1212.0]",108.0389
2,3N_R,"[10981.0, 5317.0, 4590.0]","[9104.0, 3797.0, 5866.0]","[9167.0, 3896.0, 5852.0]","[63.0, 99.0, -14.0]","[1877.0, 1520.0, -1276.0]",117.9873
19,3N_L,"[11018.0, 5260.0, 4370.0]","[9104.0, 3797.0, 5524.0]","[9203.0, 3803.0, 5597.0]","[99.0, 6.0, 74.0]","[1914.0, 1463.0, -1154.0]",123.7034


In [ ]:
plt.figure(figsize=(30, 5)) 
plt.axhline(y=rms, linestyle='--', linewidth=2, color='red', label='Mean')
plt.text(0, rms, f"Mean RMS={round(rms,2)}")
plt.bar(df['structure'], df['sumsquares'])

In [ ]:
import itk
structure = '7n_L'
file_path = f"/net/birdstore/Active_Atlas_Data/data_root/atlas_data/AtlasV8/nii/{structure}.nii"
image = itk.imread(file_path)
moments = itk.ImageMomentsCalculator.New(image)
moments.Compute()
center_of_gravity = moments.GetCenterOfGravity()
x,y,z = center_of_gravity
#print(f'Center of gravity (sitk): {(x,y,z)}')
cog = np.array((x,y,z)) * 10
com = np.loadtxt(f"/net/birdstore/Active_Atlas_Data/data_root/atlas_data/AtlasV8/com/{structure}.txt")
cog - com

In [ ]:
cog = np.array((x,y,z))
com = np.loadtxt("/net/birdstore/Active_Atlas_Data/data_root/atlas_data/AtlasV8/com/SC.txt")
cog*10 - com